In [3]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd

In [4]:
root_directory='/media/johannes/Crucial SSD/PP2CS/'
fasta_filenames ={
'test': 'ec_vs_NOec_pide20_c50_test.fasta',
'train': 'ec_vs_NOec_pide20_c50_train.fasta',
'val': 'ec_vs_NOec_pide20_c50_val.fasta'    
}

identifiers = []
data_set_seqs = {'train': [], 'val':[], 'test': []}
data_set_names = {'train': [], 'val':[], 'test': []}

for dataset in ['train','test','val']:
    with open(root_directory + fasta_filenames[dataset]) as fasta_file:  # Will close handle cleanly

        for title, sequence in SimpleFastaParser(fasta_file):
            data_set_names[dataset].append(title.split(None, 1)[0])  # First word is ID
            data_set_seqs[dataset].append(sequence)
            

In [5]:
annotations =  pd.read_csv(root_directory + 'merged_anno.txt',sep="\\t",names=['index','EC'],index_col=0)
annotations

<ipython-input-5-1f619168d6fe>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  annotations =  pd.read_csv(root_directory + 'merged_anno.txt',sep="\\t",names=['index','EC'],index_col=0)


,EC
index,
Q827Q8,6.3.5.5
Q8G830,6.5.1.2
B9DIF1,5.4.99.62
Q8CWW2,6.1.1.21
A1W892,2.7.1.130
...,...
A5CEC9,6.3.5.-
Q4QLS4,2.3.3.13
Q2YUG0,2.3.1.234


In [6]:
#Get average sequence length
avg_seqlength = {'train': 0, 'test': 0, 'val': 0}
for set in data_set_seqs:
    avg_seqlength[set] = len(''.join(data_set_seqs[set]))/float(len(data_set_seqs[set]))
avg_seqlength

{'train': 292.3255668070616, 'test': 292.322, 'val': 295.266}

In [7]:
num_enzymes = {'train': 0, 'test': 0, 'val': 0}
for set in data_set_names:
    for name in data_set_names[set]:
        if name in annotations.index:
            num_enzymes[set] += 1
num_enzymes

{'train': 5821, 'test': 91, 'val': 81}

In [8]:
for x in data_set_names:
    print ("{0} avg seq length: \t\t{1}".format(x, avg_seqlength[x]))
    print ("{0} total nr of proteins: \t{1}".format(x, len(data_set_names[x])))
    print ("{0} total nr of enzymes: \t{1}".format(x, num_enzymes[x]))
    print ("{0} enzyme pct: \t\t{1} %".format(x, num_enzymes[x]/float(len(data_set_names[x]))))

train avg seq length: 		292.3255668070616
train total nr of proteins: 	33477
train total nr of enzymes: 	5821
train enzyme pct: 		0.1738805747229441 %
val avg seq length: 		295.266
val total nr of proteins: 	500
val total nr of enzymes: 	81
val enzyme pct: 		0.162 %
test avg seq length: 		292.322
test total nr of proteins: 	500
test total nr of enzymes: 	91
test enzyme pct: 		0.182 %


In [ ]:
import numpy as np
import h5py

embeddings = dict()

with h5py.File(root_directory + 'ec_vs_NOec_pide20_c50_train.h5', 'r') as f:
    # List all groups
    #print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    for key in f.keys():
        embeddings[key] = np.array(f[key][:]) # needs about 40GB RAM